In [1]:
using Flux

In [2]:
include("../SapoDatasets.jl")

transform (generic function with 1 method)

In [3]:
il = ImageList_fromfolder("Images", 500)

ImageList(["Images/Cats/cat1.jpeg", "Images/Cats/cat2.jpeg", "Images/Cats/cat3.jpeg", "Images/Dogs/dog1.jpeg", "Images/Dogs/dog2.jpeg", "Images/Dogs/dog3.jpeg"], imagelistget, 500)

In [4]:
sil = SplitImageList_byrandpct(il)

SplitImageList(["Images/Cats/cat1.jpeg", "Images/Cats/cat2.jpeg", "Images/Cats/cat3.jpeg", "Images/Dogs/dog1.jpeg", "Images/Dogs/dog2.jpeg", "Images/Dogs/dog3.jpeg"], imagelistget, 500, [2, 3, 4, 5, 6], [1])

In [5]:
lil = LabeledImageList_fromfolder(sil)

LabeledImageList(["Images/Cats/cat1.jpeg", "Images/Cats/cat2.jpeg", "Images/Cats/cat3.jpeg", "Images/Dogs/dog1.jpeg", "Images/Dogs/dog2.jpeg", "Images/Dogs/dog3.jpeg"], Any["Cats", "Cats", "Cats", "Dogs", "Dogs", "Dogs"], imagelistget, identity, 500, [2, 3, 4, 5, 6], [1])

In [6]:
train_dset = getimage_dset(lil)

ImageDataset(["Images/Cats/cat2.jpeg", "Images/Cats/cat3.jpeg", "Images/Dogs/dog1.jpeg", "Images/Dogs/dog2.jpeg", "Images/Dogs/dog3.jpeg"], Any["Cats", "Cats", "Dogs", "Dogs", "Dogs"], imagelistget, identity, 500, "train")

In [7]:
valid_dset = getimage_dset(lil, "valid")

ImageDataset(["Images/Cats/cat1.jpeg"], Any["Cats"], imagelistget, identity, 500, "valid")

In [8]:
flatten(x) = reshape(x, 64, 1)

flatten (generic function with 1 method)

In [36]:
expand(x) = reshape(x, size(x)..., 1)

expand (generic function with 1 method)

In [27]:
model = Chain(expand, Conv((3,3), 3=>16, pad=1),Conv((3,3), 16=>32, pad=1,stride=2), Conv((3,3), 32=>64, pad=1, stride=2),
            MeanPool((125,125)), flatten, Dense(64, 2), softmax)

Chain(expand, Conv((3, 3), 3=>16), Conv((3, 3), 16=>32), Conv((3, 3), 32=>64), MeanPool((125, 125), pad = (0, 0, 0, 0), stride = (125, 125)), flatten, Dense(64, 2), softmax)

In [28]:
opt = ADAM()

ADAM(0.001, (0.9, 0.999), IdDict{Any,Any}())

In [43]:
function loss(x, y)
    ŷ = model(x)[1]
    y = y == "Cat" ? 1 : 0
    Flux.crossentropy([ŷ], [y])
end

loss (generic function with 1 method)

In [30]:
ps = params(model)

Params([Float32[0.39947978 0.18144159 0.28760168; -0.39930925 -0.4544175 -0.23888735; -0.050568778 0.3745162 -0.38684416]

Float32[0.44587994 -0.3770692 0.16985588; 0.46492523 -0.14516853 -0.45168096; 0.1205419 0.34319887 -0.4572403]

Float32[0.4617264 0.14414616 0.42353687; -0.14649749 -0.27933055 -0.46219292; 0.04583566 0.13235956 -0.14548984]

Float32[0.12320624 -0.19457701 0.3732942; -0.47885606 -0.2511358 -0.47078955; 0.24641739 0.35122272 -0.017559478]

Float32[-0.47647333 0.37737885 -0.21057893; -0.3492877 0.31341454 -0.43552917; 0.14891794 0.4055776 -0.4383478]

Float32[-0.14856532 -0.46766257 0.15783311; 0.24534772 0.030054472 -0.29910424; 0.29617906 0.10215641 -0.4581662]

Float32[-0.48408324 0.25294992 -0.119575836; 0.32431695 0.05581173 0.2007393; -0.4810045 0.28955963 -0.16429722]

Float32[0.2279234 -0.31871647 -0.05026673; -0.0969965 -0.08353381 -0.46459654; 0.46443012 -0.31192276 -0.22266525]

Float32[-0.48578668 -0.078839585 0.25488263; 0.46573126 0.03451463 0.047429405

In [44]:
Flux.train!(loss, ps, train_dset, opt)

In [50]:
x = valid_dset[1]

(Normed{UInt8,8}[0.325N0f8 0.031N0f8 … 0.137N0f8 0.792N0f8; 0.424N0f8 0.078N0f8 … 0.224N0f8 0.792N0f8; … ; 0.071N0f8 0.157N0f8 … 0.788N0f8 0.769N0f8; 0.18N0f8 0.341N0f8 … 0.827N0f8 0.784N0f8]

Normed{UInt8,8}[0.827N0f8 0.122N0f8 … 0.482N0f8 0.188N0f8; 0.776N0f8 0.208N0f8 … 0.141N0f8 0.216N0f8; … ; 0.561N0f8 0.8N0f8 … 0.729N0f8 0.957N0f8; 0.169N0f8 0.831N0f8 … 0.408N0f8 0.937N0f8]

Normed{UInt8,8}[0.957N0f8 0.482N0f8 … 0.063N0f8 0.616N0f8; 0.961N0f8 0.161N0f8 … 0.161N0f8 0.192N0f8; … ; 0.149N0f8 0.725N0f8 … 0.184N0f8 0.588N0f8; 0.294N0f8 0.4N0f8 … 0.392N0f8 0.251N0f8], "Cats")

In [51]:
model(x[1])

Tracked 2×1 Array{Float32,2}:
 0.99959844f0   
 0.00040152887f0